In [5]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn import metrics
import warnings

from sklearn.datasets import make_moons
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
filepath = "/content/gdrive/MyDrive/ColabNotebooks/iot23_combined.csv"

In [8]:
df = pd.read_csv(filepath)

In [9]:
del df['Unnamed: 0']

In [10]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0.0,0.0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1,2.998804,0.0,0.0,0.0,3.0,180.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
2,2.999300,0.0,0.0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
3,2.993548,0.0,0.0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
4,2.998807,0.0,0.0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993282,90.034713,3300.0,0.0,0.0,11.0,3608.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
1993283,90.399970,340.0,0.0,0.0,9.0,844.0,0.0,0.0,Benign,1,...,0,0,0,0,0,0,0,0,0,0
1993284,89.824030,72.0,0.0,0.0,3.0,216.0,0.0,0.0,Benign,1,...,0,0,0,0,0,0,0,0,0,0
1993285,45.215915,200.0,0.0,0.0,8.0,648.0,0.0,0.0,Benign,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df['label'].value_counts()

PartOfAHorizontalPortScan     1548914
Benign                         229946
Okiru                          100624
DDoS                           100018
C&C                              6572
Attack                           6375
C&C-HeartBeat                     751
C&C-FileDownload                   46
C&C-Torii                          16
FileDownload                       13
C&C-HeartBeat-FileDownload          8
Okiru-Attack                        3
C&C-Mirai                           1
Name: label, dtype: int64

In [12]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.15)

In [13]:
xgb = XGBClassifier(activation = 'relu', random_state=10, max_depth=10)
mlp = MLPClassifier(activation = 'relu', random_state=10, nesterovs_momentum=True, learning_rate='invscaling')
rf=RandomForestClassifier(n_estimators= 100, random_state=10, criterion = 'entropy', max_depth = 8, max_features = 'auto')
#nb = GaussianNB()

In [ ]:
voting_clf = VotingClassifier(
        estimators=[('xgb', xgb), ('mlp', mlp), ('rf', rf)], voting="hard")
voting_clf.fit(X_train, Y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(activation='relu', max_depth=10,
                                            random_state=10)),
                             ('mlp',
                              MLPClassifier(learning_rate='invscaling',
                                            random_state=10)),
                             ('rf',
                              RandomForestClassifier(criterion='entropy',
                                                     max_depth=8,
                                                     random_state=10))])

In [ ]:
for clf in (xgb, mlp, rf, voting_clf):
    clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(Y_test, y_pred))
    warnings.filterwarnings('ignore')
    print("Classifiction Report :")
    print(classification_report(Y_test, y_pred))

In [ ]:
from sklearn.datasets import make_regression
from matplotlib import pyplot

importance = xgb.feature_importances_

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
corr=df.corr()

sns.set(font_scale=2.15)
plt.figure(figsize=(70, 50))

sns.heatmap(corr, vmax=.8, linewidths=0.01,
            square=True,annot=True,cmap='YlGnBu',linecolor="black")
plt.title('Correlation between features')